In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv("Data/Unipol_dataset_lab3.csv")
data_f_dist = data[data["total_distance"] > 0.0]#distances equal to 0 are an outlier
print(data_f_dist.shape)
data_f_dist = data_f_dist[~((data_f_dist["total_distance"] <0.1) & (data_f_dist["road"] == "A"))]#removes 50ish trips


In [ ]:
grouped = data_f_dist.groupby("road")
def ecdf(data):
    x = np.sort(data)
    y = np.arange(1, len(x)+1) / len(x)
    return x, y

plt.figure(figsize=(10, 6))

for road, group in grouped:
    x, y = ecdf(group["total_distance"])
    plt.semilogx(x, y, label=f"Road {road}")

plt.xlabel("Trip Distance (Km)")
plt.ylabel("ECDF")
plt.title("ECDF of Trip Distances for Different Road Types")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:

data_f_dist["start_time"] = pd.to_datetime(data_f_dist["start_time"])
data_f_dist["stop_time"] = pd.to_datetime(data_f_dist["stop_time"])
data_f_dist["trip_duration"] = (data_f_dist["stop_time"] - data_f_dist["start_time"]).dt.total_seconds() / 60.0 
grouped = data_f_dist.groupby("road")

plt.figure(figsize=(10, 6))
for road, group in grouped:
    x, y = ecdf(group["trip_duration"])
    plt.semilogx(x, y, label=f"Road {road}")

plt.xlabel("Trip Duration (minutes)")
plt.ylabel("ECDF")
plt.title("ECDF of Trip Durations for Different Road Types")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
data_f_dist_avg = data_f_dist[~((data_f_dist["total_distance"] / (data_f_dist["trip_duration"] / 60) <= 10) & (data_f_dist["trip_duration"] > 600))]#removes 100ish trips
data_f_dist_avg = data_f_dist[~((data_f_dist["total_distance"] / (data_f_dist["trip_duration"] / 60) <= 5) & (data_f_dist["trip_duration"] > 300))]#removes 50ish trips

data_f_dist_avg_dur = data_f_dist_avg[data_f_dist_avg['trip_duration'] < 1000]#remove duration grater than 1000 minutes
plt.figure(figsize=(10, 6))
for road_type, group in data_f_dist_avg_dur.groupby("road"):
    plt.scatter(group["trip_duration"], group["total_distance"], label=f"Road {road_type}", alpha=0.7)

plt.xlabel("Trip Duration (minutes)")
plt.ylabel("Total Distance (km)")
plt.title("Relationship Between Trip Duration and Total Distance by Road Type")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
#no filter

data_f_dist = data_f_dist[data_f_dist['trip_duration'] < 100000]#remove duration grate than 24 hours
plt.figure(figsize=(10, 6))
for road_type, group in data_f_dist.groupby("road"):
    plt.scatter(group["trip_duration"], group["total_distance"], label=f"Road {road_type}", alpha=0.7)

plt.xlabel("Trip Duration (minutes)")
plt.ylabel("Total Distance (km)")
plt.title("Relationship Between Trip Duration and Total Distance by Road Type")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
nonFiltered_count = data.groupby(["vehicle_id", "trip_id"]).nunique().shape[0]
print(f"Number of trips before filtering: {nonFiltered_count}")

In [ ]:
fil_dist_avg_dur_count = data_f_dist_avg_dur.groupby(["vehicle_id", "trip_id"]).nunique().shape[0]
print(f"Number of trips after filtering for average distance and duration: {fil_dist_avg_dur_count}")

In [ ]:
data_f_dist.groupby(["vehicle_id", "trip_id"]).nunique().shape[0]


# Task 1

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv("Data/Unipol_dataset_lab3.csv")
data = data[data["total_distance"] > 0.0]
data = data[~((data["total_distance"] <0.1) & (data["road"] == "A"))]

data["start_time"] = pd.to_datetime(data["start_time"])
data["stop_time"] = pd.to_datetime(data["stop_time"])
data["trip_duration"] = (data["stop_time"] - data["start_time"]).dt.total_seconds() / 60.0 #convert to minutes

data = data[~((data["total_distance"] / (data["trip_duration"] / 60) <= 10) & (data["trip_duration"] > 600))]#removes 100ish trips
data = data[~((data["total_distance"] / (data["trip_duration"] / 60) <= 5) & (data["trip_duration"] > 300))]#removes 50ish trips
data = data[data['trip_duration'] < 1200]#remove duration grate than 1200 minutes, 20 hours

#Number of trips before filtering: 966000 
print(data.shape[0], data.groupby(["vehicle_id", "trip_id"]).nunique().shape[0])

In [ ]:
data["date"] = data["start_time"].dt.date
aggregated = data.groupby(["vehicle_id", "date"]).agg(
    number_of_trips=("trip_id", "nunique"),
    total_travel_distance=("total_distance", "sum"),
    total_driving_time=("trip_duration", "sum")
).reset_index()
aggregated["utilization_percentage"] = (aggregated["total_driving_time"] / (24 * 60) * 100)  # Fraction of driving time in 24 hours

aggregated["day_of_week"] = pd.to_datetime(aggregated["date"]).dt.dayofweek
aggregated['day_type'] = aggregated['day_of_week'].apply(lambda x: 'workday' if x < 5 else 'weekend/holiday')

stats = aggregated.groupby(['vehicle_id', 'day_type']).agg({
    'number_of_trips': ['mean', 'median', 'std'],
    'total_travel_distance': ['mean', 'median', 'std'],
    'utilization_percentage': ['mean', 'median', 'std']
}).reset_index()

overall_stats = aggregated.groupby('day_type').agg({
    'number_of_trips': ['mean', 'median', 'std'],
    'total_travel_distance': ['mean', 'median', 'std'],
    'utilization_percentage': ['mean', 'median', 'std']
}).reset_index()

print(overall_stats)


In [ ]:
from datetime import timedelta

# Function to split trips that span multiple days
def split_trip(row):
    trips = []
    start_time = row['start_time']
    stop_time = row['stop_time']
    total_distance = row['total_distance']
    trip_duration = row['trip_duration']
    
    
    while start_time.date() != stop_time.date():
        end_of_day = start_time.replace(hour=23, minute=59, second=59)
        duration = (end_of_day - start_time).total_seconds() / 60.0
        proportion = duration / trip_duration
        distance = total_distance * proportion
        
        trips.append({
            'vehicle_id': row['vehicle_id'],
            'trip_id': row['trip_id'],
            'start_time': start_time,
            'stop_time': end_of_day,
            'road': row['road'],
            'total_distance': distance,
            'trip_duration': duration,
            'date': start_time.date()
        })
        
        start_time = end_of_day + timedelta(seconds=1)
        trip_duration -= duration
        total_distance -= distance
        
        duration = (stop_time - start_time).total_seconds() / 60.0
        proportion = duration / trip_duration
        distance = total_distance * proportion
        
        trips.append({
            'vehicle_id': row['vehicle_id'],
            'trip_id': row['trip_id'],
            'start_time': start_time,
            'stop_time': stop_time,
            'road': row['road'],
            'total_distance': distance,
            'trip_duration': duration,
            'date': start_time.date()
        })
    
    return trips

split_trips = []
for _, row in data.iterrows():
    if row['start_time'].date() != row['stop_time'].date():        
        split_trips.extend(split_trip(row))
    else:
        split_trips.append(row.to_dict())

split_data = pd.DataFrame(split_trips)
split_data.shape[0]


In [ ]:
split_data["date"] = split_data["start_time"].dt.date
aggregated = split_data.groupby(["vehicle_id", "date"]).agg(
    number_of_trips=("trip_id", "nunique"),
    total_travel_distance=("total_distance", "sum"),
    total_driving_time=("trip_duration", "sum")
).reset_index()
aggregated["utilization_percentage"] = (aggregated["total_driving_time"] / (24 * 60) * 100)  # Fraction of driving time in 24 hours

aggregated["day_of_week"] = pd.to_datetime(aggregated["date"]).dt.dayofweek
aggregated['day_type'] = aggregated['day_of_week'].apply(lambda x: 'workday' if x < 5 else 'weekend/holiday')

stats = aggregated.groupby(['vehicle_id', 'day_type']).agg({
    'number_of_trips': ['mean', 'median', 'std'],
    'total_travel_distance': ['mean', 'median', 'std'],
    'utilization_percentage': ['mean', 'median', 'std']
}).reset_index()

overall_stats = aggregated.groupby('day_type').agg({
    'number_of_trips': ['mean', 'median', 'std'],
    'total_travel_distance': ['mean', 'median', 'std'],
    'utilization_percentage': ['mean', 'median', 'std']
}).reset_index()
print("Statistics with splitted trips")
print(overall_stats)



In [ ]:
import seaborn as sns
hist_data = aggregated[aggregated['number_of_trips'] < 80]
hist_data = hist_data[hist_data['utilization_percentage'] <= 100]
hist_data = hist_data[hist_data['total_travel_distance'] <= 1200]
sns.histplot(data=hist_data, x='number_of_trips', hue='day_type', kde=True)
plt.title('Distribution of Number of Trips (Workdays vs. Weekends/Holidays)')
plt.xlabel('Number of Trips')
plt.legend(title='Day Type', labels=[ 'Weekend/Holiday', 'Workday'])
plt.show()

In [ ]:
sns.histplot(data=hist_data, x='total_travel_distance', hue='day_type', kde=True)
plt.title('Distribution of Total Travel Distance (Workdays vs. Weekends/Holidays)')
plt.xlabel('Total Travel Distance (km)')
plt.ylabel('Count')
plt.legend(title='Day Type', labels=[ 'Weekend/Holiday', 'Workday'])
plt.show()

In [ ]:
sns.histplot(data=hist_data, x='utilization_percentage', hue='day_type', kde=True)
plt.title('Distribution of Utilization Percentage (Workdays vs. Weekends/Holidays)')
plt.show()

In [16]:
avg_utilization = hist_data.groupby(['vehicle_id', 'day_type'])['utilization_percentage'].mean().unstack()

avg_utilization['difference'] = abs(avg_utilization['workday'] - avg_utilization['weekend/holiday'])

In [ ]:

plt.figure(figsize=(10, 6))
sns.histplot(avg_utilization['difference'], bins=50, kde=True, color='skyblue')

plt.title('Distribution of Utilization Difference Between Workdays and Weekends')
plt.xlabel('Difference in Utilization Percentage (%)')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

In [ ]:

import seaborn as sns
vehicles = data[data["vehicle_id"].isin(data["vehicle_id"].unique())] #[:1000]
vehicles = vehicles[vehicles["vehicle_id"] != 539]

road_type_counts = vehicles.groupby(["vehicle_id", "road"]).size().unstack(fill_value=0)
road_type_counts = road_type_counts[(road_type_counts.sum(axis=1) > 0)]
road_type_fractions = road_type_counts.div(road_type_counts.sum(axis=1), axis=0)

road_type_fractions['Dominant_Road'] = road_type_fractions.idxmax(axis=1)
road_type_fractions = road_type_fractions.sort_values(by='Dominant_Road').drop(columns='Dominant_Road')

plt.figure(figsize=(16, 10))
sns.heatmap(
    road_type_fractions,
    cmap="Blues",
    cbar_kws={'label': 'Fraction of Trips'},
    xticklabels=True,
    yticklabels=False
)

plt.title("Heatmap of Fraction of Trips by Road Type for Each Vehicle")
plt.xlabel("Road Type")
plt.ylabel("Vehicle Index")
plt.show()


road_type_averages = road_type_fractions.mean().sort_values(ascending=False)

plt.figure(figsize=(10, 6))
road_type_averages.plot(kind='bar', color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
plt.title("Average Fraction of Trips by Road Type")
plt.ylabel("Fraction")
plt.xlabel("Road Type")
plt.xticks(rotation=0)
plt.show()


In [ ]:
road_type_fractions = road_type_counts.div(road_type_counts.sum(axis=1), axis=0)

plt.figure(figsize=(12, 8))
colors = sns.color_palette("viridis", len(road_type_fractions.columns))

for road_type, color in zip(road_type_fractions.columns, colors):
    sns.kdeplot(road_type_fractions[road_type], label=f"Road {road_type}", shade=True, color=color)

plt.xlabel("Fraction of Trips")
plt.ylabel("Density")
plt.title("PDF of Fraction of Trips for Each Road Type")
plt.legend(title="Road Type")
plt.grid(True)
plt.show()

In [ ]:
road_type_distance = vehicles.groupby(["vehicle_id", "road"])["total_distance"].sum().unstack(fill_value=0)

road_type_distance_fractions = road_type_distance.div(road_type_distance.sum(axis=1), axis=0)

plt.figure(figsize=(12, 8))
colors = sns.color_palette("viridis", len(road_type_distance_fractions.columns))
for road_type, color in zip(road_type_distance_fractions.columns, colors):
    sns.kdeplot(road_type_distance_fractions[road_type], label=f"Road {road_type}", fill=True, color=color)

plt.xlabel("Fraction of Total Travel Distance")
plt.ylabel("Density")
plt.title("PDF of Fraction of Total Travel Distance for Each Road Type")
plt.legend(title="Road Type")
plt.grid(True)
plt.show()

In [ ]:
road_type_counts = vehicles.groupby(["vehicle_id", "road"]).size().unstack(fill_value=0)
road_type_counts = road_type_counts[30:40]

fig, ax = plt.subplots(figsize=(12, 8))

road_types = road_type_counts.columns
bottom = np.zeros(len(road_type_counts))

for road_type in road_types:
    if road_type == "_":
        label = "Unknown"
    else:
        label = road_type
    ax.bar(road_type_counts.index, road_type_counts[road_type] / road_type_counts.sum(axis=1), bottom=bottom, label=label)
    bottom += road_type_counts[road_type] / road_type_counts.sum(axis=1)

ax.set_title("Fraction of Trips with Different Road Types for Vehicles 30-40")
ax.set_ylabel("Fraction of Trips")
ax.set_xlabel("Vehicle ID")
ax.set_xticks(road_type_counts.index)
ax.legend(title="Road Type")
plt.tight_layout()
plt.show()

In [ ]:
average_time_fractions = road_type_distance_fractions.mean()

plt.figure(figsize=(10, 6))
average_time_fractions.plot(kind='bar', color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
plt.title("Average Fraction of Time Spent on Each Road Type")
plt.ylabel("Average Fraction of Time")
plt.xlabel("Road Type")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.cluster import KMeans

# Example: cluster vehicles based on average # of trips/day, avg distance/day, fraction of trips on road 'A', etc.
vehicle_agg = (
    data.groupby("vehicle_id")
    .agg(
        avg_trips_per_day=("trip_id", lambda x: x.nunique() / ( (data["date"].max() - data["date"].min()).days + 1 )),
        avg_distance_per_day=("total_distance", lambda x: x.sum()   / ( (data["date"].max() - data["date"].min()).days + 1 )),
        fraction_road_A=("road", lambda x: np.mean(x == "A")),
        fraction_road_U=("road", lambda x: np.mean(x == "U")),
        fraction_road_E=("road", lambda x: np.mean(x == "E")),
    )
)

# K-Means with 3 clusters
kmeans = KMeans(n_clusters=3, random_state=42)
vehicle_agg["cluster"] = kmeans.fit_predict(vehicle_agg)

print(vehicle_agg.head(10))


In [ ]:
cluster_names = {
    0: "Urban Short Trips",
    1: "Highway Long Distances",
    2: "Frequent Commuters"
}
print("Cluster centers:")
print(kmeans.cluster_centers_)
cluster_profiles = vehicle_agg.groupby("cluster").mean()
print(cluster_profiles)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
colors = {0: "red", 1: "green", 2: "blue"}  

for cluster_id in vehicle_agg["cluster"].unique():
    subset = vehicle_agg[vehicle_agg["cluster"] == cluster_id]
    ax.scatter(
        subset["avg_trips_per_day"],
        subset["avg_distance_per_day"],
        c=colors[cluster_id],
        label=cluster_names[cluster_id],
        alpha=0.6
    )

ax.set_xlabel("Average Trips per Day")
ax.set_ylabel("Average Distance per Day (km)")
ax.set_title("K-Means Clusters (2D View)")
ax.legend()
plt.show()

# Task 2

1. Tesla model 3 range: 420km
2. BMW iX xDrive40 range: 360km
3. Citroen e-C3  range: 260km



Evaluation Metrics
1. Percentage of Feasible Trips 
2. Energy Consumption per 100 km
3. Charging time

In [27]:
import json

cars = {
  "cars": {
    "Tesla Model 3": {
      "cost": 40990, #EUR Germany
      "range_km": 420,
      "evaluation_metrics": {
        "energy_consumption_per_100_km": 137, #Wh/km
        "charging_time": 6
      },
      "performance": {
        "acceleration_0_100_kmh": "6.1 sec",
        "top_speed_kmh": 201,
        "total_power_kw": 208,
        "total_torque_nm": 420,
        "drive": "Rear"
      },
      "battery": {
        "nominal_capacity_kwh": 60.0,
        "useable_capacity_kwh": 57.5,
        "battery_type": "Lithium-ion"
      },
      "charging": {
        "home_destination": {
          "charge_port": "Type 2",
          "port_location": "Left Side - Rear",
          "charge_power_kw_ac": 11,
          "charge_time_h": "6h15m",
          "charge_speed_kmh": 68
        },
        "fast_charging": {
          "charge_port": "CCS",
          "port_location": "Left Side - Rear",
          "charge_power_max_kw_dc": 170,
          "charge_power_10_80_kw_dc": 108,
          "charge_time_min": 24,
          "charge_speed_kmh": 730
        }
      },
      "consumption": {
        "city_cold_weather_wh_km": 149,
        "highway_cold_weather_wh_km": 189,
        "combined_cold_weather_wh_km": 167,
        "city_mild_weather_wh_km": 93,
        "highway_mild_weather_wh_km": 142,
        "combined_mild_weather_wh_km": 116
      }
    },
    "BMW iX xDrive40": {
      "cost": 77300, #EUR Germany
      "range_km": 360,
      "evaluation_metrics": {
        "energy_consumption_per_100_km": 200, #Wh/km
        "charging_time": 8
      },
      "performance": {
        "acceleration_0_100_kmh": "6.1 sec",
        "top_speed_kmh": 200,
        "total_power_kw": 240,
        "total_torque_nm": 630,
        "drive": "All-Wheel Drive"
      },
      "battery": {
        "nominal_capacity_kwh": 76.6,
        "useable_capacity_kwh": 71,
        "battery_type": "Lithium-ion"
      },
      "charging": {
        "home_destination": {
          "charge_port": "Type 2",
          "port_location": "Left Side - Rear",
          "charge_power_kw_ac": 11,
          "charge_time_h": "8h",
          "charge_speed_kmh": 45
        },
        "fast_charging": {
          "charge_port": "CCS",
          "port_location": "Left Side - Rear",
          "charge_power_max_kw_dc": 150,
          "charge_power_10_80_kw_dc": 110,
          "charge_time_min": 31,
          "charge_speed_kmh": 580
        }
      },
      "consumption": {
        "city_cold_weather_wh_km": 180,
        "highway_cold_weather_wh_km": 220,
        "combined_cold_weather_wh_km": 200,
        "city_mild_weather_wh_km": 140,
        "highway_mild_weather_wh_km": 180,
        "combined_mild_weather_wh_km": 160
      }
    },
    "Citroen e-C3": {
      "cost": 23300, #EUR Germany
      "range_km": 260,
      "evaluation_metrics": {
        "energy_consumption_per_100_km": 150, #Wh/km
        "charging_time": 7
      },
      "performance": {
        "acceleration_0_100_kmh": "9.7 sec",
        "top_speed_kmh": 150,
        "total_power_kw": 100,
        "total_torque_nm": 260,
        "drive": "Front"
      },
      "battery": {
        "nominal_capacity_kwh": 50,
        "useable_capacity_kwh": 45,
        "battery_type": "Lithium-ion"
      },
      "charging": {
        "home_destination": {
          "charge_port": "Type 2",
          "port_location": "Left Side - Rear",
          "charge_power_kw_ac": 7.4,
          "charge_time_h": "7h",
          "charge_speed_kmh": 37
        },
        "fast_charging": {
          "charge_port": "CCS",
          "port_location": "Left Side - Rear",
          "charge_power_max_kw_dc": 100,
          "charge_power_10_80_kw_dc": 75,
          "charge_time_min": 30,
          "charge_speed_kmh": 520
        }
      },
      "consumption": {
        "city_cold_weather_wh_km": 160,
        "highway_cold_weather_wh_km": 200,
        "combined_cold_weather_wh_km": 180,
        "city_mild_weather_wh_km": 120,
        "highway_mild_weather_wh_km": 160,
        "combined_mild_weather_wh_km": 140
      }
    }
  }
}



task 3

+-------------------+       +-------------------+       +-------------------+
| Define EVs & Trips| ----> | Replicate Trips   | ----> | Generate Report   |
+-------------------+       +-------------------+       +-------------------+
                               |
                               v
                        +-------------------+
                        | Simulate Consumption |
                        +-------------------+

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

data["start_time"] = pd.to_datetime(data["start_time"])
data["stop_time"] = pd.to_datetime(data["stop_time"])

data["trip_duration"] = (data["stop_time"] - data["start_time"]).dt.total_seconds() / 60.0
weather = "mild" #mild or cold
charging_type = "home_destination" #fast_charging or home_destination
vehicle_data = data.groupby("vehicle_id")
price_fast_charging = 0.5 #EUR per kWh
price_home_charging = 0.15 #EUR per kWh

results_list = []  # Initialize results_list outside the loop for each car model

for car, specs in cars["cars"].items():
    total_trips = 0
    feasible_trips = 0
    non_feasible = 0
    energy_consumed_100km = 0
    charge = 0
    kwh_after_trip = []
    charging_time = []
    energy_consumed = []
    distance_trips = []
    distance_vehicles = []
    energy_vehicles = []
    battery_capacity = specs["battery"]["useable_capacity_kwh"]   #  kWh 
    consumption_per_100_km = specs["evaluation_metrics"]["energy_consumption_per_100_km"]
    last_stop = ""

    for vehicle_id, trips in vehicle_data:
        remaining_kwh = battery_capacity
        total_charge_time = 0
        distance_vehicle = []
        energy_vehicle = []
        for _, trip in trips.iterrows():
            trip_duration_hours = trip["trip_duration"] / 60.0  
            total_trips += 1
            distance_km = trip["total_distance"]
            if last_stop != "":
                time_since_last_stop = (trip["start_time"] - last_stop).total_seconds()/60 #minutes 
                if time_since_last_stop > 20:
                    
                    if charging_type == "fast_charging":
                        charged_power = specs["charging"][charging_type]["charge_power_10_80_kw_dc"] * time_since_last_stop / 60 #max or not?
                        if remaining_kwh + charged_power <= battery_capacity:
                            remaining_kwh = remaining_kwh + charged_power
                            charge_time= time_since_last_stop/60
                        else:
                            charge_time = (battery_capacity - remaining_kwh)/specs["charging"][charging_type]["charge_power_10_80_kw_dc"]
                            remaining_kwh = battery_capacity
                    else:
                        charged_power = specs["charging"]["home_destination"]["charge_power_kw_ac"] * time_since_last_stop / 60
                        if remaining_kwh + charged_power <= battery_capacity:
                            remaining_kwh = remaining_kwh + charged_power
                            charge_time = time_since_last_stop/60
                        else:
                            charge_time = (battery_capacity - remaining_kwh)/specs["charging"]["home_destination"]["charge_power_kw_ac"]
                            remaining_kwh = battery_capacity
                    charging_time.append(charge_time)
                    total_charge_time += charge_time
                    
            if weather == "cold":
                if trip.road == "U":
                    consumption = specs["consumption"]["city_cold_weather_wh_km"]
                elif trip.road == "A":
                    consumption = specs["consumption"]["highway_cold_weather_wh_km"]
                else:
                    consumption = specs["consumption"]["combined_cold_weather_wh_km"]
            else:
                if trip.road == "U":
                    consumption = specs["consumption"]["city_mild_weather_wh_km"]
                elif trip.road == "A":
                    consumption = specs["consumption"]["highway_mild_weather_wh_km"]
                else:
                    consumption = specs["consumption"]["combined_mild_weather_wh_km"]

            energy_needed = (distance_km * consumption) / 1000  #  Wh in kWh
            if energy_needed > remaining_kwh:
                non_feasible = non_feasible + 1
                remaining_kwh = 0
            else:
                remaining_kwh = remaining_kwh - energy_needed
                feasible_trips = feasible_trips + 1
                energy_consumed.append(energy_needed)
                energy_vehicle.append(energy_needed)
                distance_trips.append(distance_km)
                distance_vehicle.append(distance_km)

            kwh_after_trip.append(remaining_kwh)
            last_stop = trip.stop_time
        distance_vehicles.append(sum(distance_vehicle))
        energy_vehicles.append(sum(energy_vehicle))
    
        feasible_percentage = ((feasible_trips - non_feasible) / total_trips) * 100 if total_trips > 0 else 0
        charge_time = np.mean(total_charge_time) 
        hours, minutes = divmod(charge_time, 60)  # hours and minutes
        avg_kwh = np.mean(kwh_after_trip)
        energy_consumed_100km = np.mean(energy_consumed)/np.mean(distance_trips)*1000
        results_list.append({
            "vehicle_id": vehicle_id,
            "car": car,
            "total_trips": total_trips,
            "feasible_percentage": feasible_percentage,
            "average_charge_time": f"{int(hours)} hours and {int(minutes)} minutes",
            "average_soc_after_trip": avg_kwh,
            "soc_percentage": avg_kwh / battery_capacity * 100,
            "energy_consumption_per_100km": energy_consumed_100km,
            "average_energy_cost": np.mean(energy_vehicles) * price_home_charging,
            "car_cost": specs['cost']
        
    })

results_df = pd.DataFrame(results_list)
print(results_df)

In [ ]:
merged_df = results_df.merge(
    vehicle_agg[["cluster"]], 
    how="left",
    left_on="vehicle_id",
    right_index=True
)

merged_df.head(10)

In [ ]:
def convert_time_to_hours(time_str):
    if pd.isna(time_str):
        return 0  
    hours, minutes = map(int, time_str.replace(" hours", "").replace(" minutes", "").split(" and "))
    return hours + minutes / 60

merged_df["avg_charging_time_hrs"] = merged_df["average_charge_time"].apply(convert_time_to_hours)


results_with_clusters = merged_df.groupby(["cluster", "car","vehicle_id"]).agg({
    "feasible_percentage": "mean",
    "avg_charging_time_hrs": "mean",
    "total_trips": "mean",
    "average_soc_after_trip": "mean",
    "energy_consumption_per_100km": "mean",
    "average_energy_cost": "mean",
    "car_cost": "mean",
}).reset_index()

print(results_with_clusters)



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.barplot(
    data=results_with_clusters, 
    x="cluster", 
    y="feasible_percentage", 
    hue="car"
)
plt.title("Feasible Trip Percentage by Cluster & EV Model")
plt.xlabel("Cluster")
plt.ylabel("Feasible Trip Percentage (%)")
plt.legend(title="Car Model")
plt.show()

In [ ]:
def calculate_compatibility_score(results_df, cars, weights_perf=(0.4, 0.3, 0.3), weights_cost=(0.6, 0.4), alpha=0.7):
    """
    Calculate the compatibility score for each EV model and vehicle_id based on performance and costs.
    
    Args:
    - results_df: DataFrame containing the results of the simulation (metrics per EV model and vehicle).
    - cars: Dictionary containing the EV purchase costs.
    - weights_perf: Weights for performance metrics (feasible trips %, charge time, energy consumption).
    - weights_cost: Weights for cost metrics (energy cost, purchase cost).
    - alpha: Weight for performance vs cost trade-off (0.7 means 70% performance, 30% cost).
    
    Returns:
    - DataFrame with compatibility scores for each vehicle and EV model.
    """
    
    results_df["feasible_percentage"] = results_df["feasible_percentage"] / 100  
    results_df["Normalized Charge Time"] = 1 - (results_df["avg_charging_time_hrs"] / results_df["avg_charging_time_hrs"].max())
    results_df["Normalized Energy Consumption"] = 1 - (results_df["energy_consumption_per_100km"] / results_df["energy_consumption_per_100km"].max())
    

    results_df["Performance Score"] = (
        weights_perf[0] * results_df["feasible_percentage"] +
        weights_perf[1] * results_df["Normalized Charge Time"] +
        weights_perf[2] * results_df["Normalized Energy Consumption"]
    )
    

    purchase_costs = {car: specs["cost"] for car, specs in cars["cars"].items()}
    results_df["car_cost"] = results_df["car"].map(purchase_costs)
    
    
    results_df["Normalized Energy Cost"] = 1 - (results_df["average_energy_cost"] / results_df["average_energy_cost"].max())
    results_df["Normalized Purchase Cost"] = 1 - (results_df["car_cost"] / results_df["car_cost"].max())
    
    
    results_df["Cost Score"] = (
        weights_cost[0] * results_df["Normalized Energy Cost"] +
        weights_cost[1] * results_df["Normalized Purchase Cost"]
    )
    
    
    results_df["Compatibility Score"] = (
        alpha * results_df["Performance Score"] +
        (1 - alpha) * results_df["Cost Score"]
    )
    
    
    sorted_results = results_df.sort_values(by=["vehicle_id", "Compatibility Score"], ascending=[True, False])
    
    return sorted_results


In [ ]:
# Calculate compatibility scores
sorted_results = calculate_compatibility_score(results_with_clusters, cars, alpha=0.7)

sorted_results.groupby("vehicle_id").head(1)


In [ ]:

def best_ev_model_per_vehicle(sorted_results):
    best_ev_df = sorted_results.groupby("vehicle_id").apply(lambda x: x.nlargest(1, "Compatibility Score")).reset_index(drop=True)
    best_ev_df = best_ev_df[["vehicle_id", "Compatibility Score", "car", "cluster"]]
    return best_ev_df

# Plot the distribution of the best EV models
def plot_best_ev_models(best_ev_df):
    import seaborn as sns
    import matplotlib.pyplot as plt

    plt.figure(figsize=(10, 6))
    sns.countplot(data=best_ev_df, x="car", order=best_ev_df["car"].value_counts().index, palette="viridis")
    plt.title("Distribution of Best EV Model for Each Vehicle ID")
    plt.xlabel("EV Model")
    plt.ylabel("Number of Vehicles")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Apply the functions
best_ev_df = best_ev_model_per_vehicle(sorted_results)
print(best_ev_df)
plot_best_ev_models(best_ev_df)


In [ ]:

cluster_ev_summary = best_ev_df.groupby(["cluster", "car"]).size().reset_index(name="Count")


def plot_cluster_ev_summary(cluster_ev_summary):
    import seaborn as sns
    import matplotlib.pyplot as plt

    plt.figure(figsize=(12, 8))
    sns.barplot(data=cluster_ev_summary, x="cluster", y="Count", hue="car", palette="tab10")
    plt.title("EV Model Distribution Across Clusters")
    plt.xlabel("Cluster (Driving Behavior)")
    plt.ylabel("Number of Vehicles")
    plt.tight_layout()
    plt.show()

plot_cluster_ev_summary(cluster_ev_summary)


## Task 6

In [ ]:
import random
import pandas as pd


def is_nighttime(dt, night_start=22, night_end=6):
    hour = dt.hour
    return (hour >= night_start) or (hour < night_end)

def simulate_all_ev_models_with_refined_policy_debug(vehicle_data, cars, min_parking_time=40):
    results = []  # Store results for each EV

    for car, specs in cars["cars"].items():
        print(f"Running simulation for {car}...\n{'-' * 60}")

        for vehicle_id, trips in vehicle_data:
            battery_capacity = specs["battery"]["useable_capacity_kwh"]  # in kWh
            price_home_charging = 0.15  
            price_fast_charging = 0.50  

            soc = battery_capacity  
            total_cost = 0  
            feasible_trips = 0
            unfeasible_trips = 0
            total_energy_consumed = 0  
            total_trip_distance = 0  
            total_charge_time = 0  
            charging_events = 0  
            soc_after_trips = []
            last_stop = None

            for trip_index, trip in trips.iterrows():
                trip_start = trip["start_time"]
                trip_stop = trip["stop_time"]
                distance_km = trip["total_distance"]
                parking_time_minutes = (trip_start - last_stop).total_seconds() / 60 if last_stop else 0

                # Charge if idle time is above the threshold
                if last_stop is not None and parking_time_minutes >= min_parking_time:
                    is_night = is_nighttime(last_stop)
                    if is_night:
                        charge_power = specs["charging"]["home_destination"]["charge_power_kw_ac"]
                        charge_cost = price_home_charging
                    else:
                        road_type = trip["road"]
                        charge_power = 0
                        if road_type == "A" and random.random() < 0.8:
                            charge_power = specs["charging"]["fast_charging"]["charge_power_max_kw_dc"]
                            charge_cost = price_fast_charging
                        elif road_type == "U" and random.random() < 0.5:
                            charge_power = specs["charging"]["fast_charging"]["charge_power_max_kw_dc"]
                            charge_cost = price_fast_charging
                        elif road_type == "E" and random.random() < 0.3:
                            charge_power = specs["charging"]["fast_charging"]["charge_power_max_kw_dc"]
                            charge_cost = price_fast_charging

                    hours_to_charge = parking_time_minutes / 60
                    energy_to_charge = charge_power * hours_to_charge

                    if soc + energy_to_charge > battery_capacity:
                        energy_to_charge = battery_capacity - soc

                    if energy_to_charge > 0:
                        soc += energy_to_charge
                        total_cost += energy_to_charge * charge_cost
                        total_charge_time += hours_to_charge
                        charging_events += 1  

                # Attempt the trip
                consumption_rate = specs["evaluation_metrics"]["energy_consumption_per_100_km"] / 1000  # kWh/km
                energy_needed = distance_km * consumption_rate

                if energy_needed > soc:
                    unfeasible_trips += 1
                    soc = 0  # Vehicle arrives with 0 SoC
                    total_energy_consumed += energy_needed
                else:
                    soc -= energy_needed
                    feasible_trips += 1
                    total_trip_distance += distance_km
                    total_energy_consumed += energy_needed

                soc_after_trips.append(soc)
                last_stop = trip_stop

            # Calculate final metrics 
            feasible_percentage = (feasible_trips / len(trips)) * 100 if len(trips) > 0 else 0
            num_days = trips["start_time"].dt.date.nunique()  # Count unique days
            avg_charge_time = total_charge_time / charging_events if charging_events > 0 else 0  
            avg_charge_time_per_day = total_charge_time / num_days if num_days > 0 else 0  
            hours_session, minutes_session = divmod(avg_charge_time * 60, 60)  
            hours_day, minutes_day = divmod(avg_charge_time_per_day * 60, 60)  
            avg_soc_after_trip = sum(soc_after_trips) / len(soc_after_trips) if len(soc_after_trips) > 0 else 0
            avg_energy_consumed_per_100km = (total_energy_consumed / total_trip_distance) * 1000 if total_trip_distance > 0 else 0

            results.append({
                "car": car,
                "vehicle_id": vehicle_id,
                "feasible_percentage": feasible_percentage,
                "avg_charging_time_hrs": f"{int(hours_session)}h {int(minutes_session)}m",
                "Average Charge Time per Day (hours:minutes)": f"{int(hours_day)}h {int(minutes_day)}m",
                "average_soc_after_trip": avg_soc_after_trip,
                "energy_consumption_per_100_km": avg_energy_consumed_per_100km,
                "average_energy_cost": total_cost,
                "Total Charging Events": charging_events,
                "car_cost": specs["cost"]
            })

    results_df = pd.DataFrame(results)
    print(results_df)
    return results_df



In [ ]:
vehicle_data = data.groupby("vehicle_id")
results_df_t6 = simulate_all_ev_models_with_refined_policy_debug(vehicle_data, cars)

In [ ]:
merged_df_t6 = results_df_t6.merge(
    vehicle_agg[["cluster"]],  
    how="left",
    left_on="vehicle_id",
    right_index=True
)


merged_df_t6.head(10)


In [ ]:
def convert_time_to_hours(time_str):
    if pd.isna(time_str):
        return 0  # Handle NaN values
    if isinstance(time_str, str):
        hours, minutes = map(int, time_str.replace("h", "").replace("m", "").split(" "))
        return hours + minutes / 60
    return time_str  


merged_df_t6["avg_charging_time_hrs"] = merged_df_t6["avg_charging_time_hrs"].apply(convert_time_to_hours)


results_with_clusters_t6 = merged_df_t6.groupby(["cluster", "car","vehicle_id"]).agg({
    "feasible_percentage": "mean",
    "avg_charging_time_hrs": "mean",
    #"total_trips": "mean",
    "average_soc_after_trip": "mean",
    "energy_consumption_per_100_km": "mean",
    "average_energy_cost": "mean",
    "car_cost": "mean",
}).reset_index()

print(results_with_clusters_t6)

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(
    data=results_with_clusters_t6, 
    x="cluster", 
    y="feasible_percentage", 
    hue="car"
)
plt.title("Feasible Trip Percentage by Cluster & EV Model")
plt.xlabel("Cluster")
plt.ylabel("Feasible Trip Percentage (%)")
plt.legend(title="Car Model")
plt.show()

In [ ]:
sorted_results_t6 = calculate_compatibility_score(results_with_clusters_t6.rename(columns={'energy_consumption_per_100_km': 'energy_consumption_per_100km'}), cars, alpha=0.7)

# Display top EV model per vehicle
sorted_results_t6.groupby("vehicle_id").head(1)

In [ ]:
best_ev_df_t6 = best_ev_model_per_vehicle(sorted_results_t6)
print(best_ev_df_t6)
plot_best_ev_models(best_ev_df_t6)

In [ ]:
cluster_ev_summary_t6 = best_ev_df_t6.groupby(["cluster", "car"]).size().reset_index(name="Count")
plot_cluster_ev_summary(cluster_ev_summary_t6)